In [85]:
# Load packages
import sys
import numpy as np
import pandas as pd
# import geopandas as gpd
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn
import xgboost as xgb
import datetime
import random
import time

Needed `pip install xlrd`

In [6]:
path = '../data/'
data_dict = pd.read_excel(path + '20200317_acaps_covid-19_goverment_measures_dataset_0.xlsx', sheet_name=1)
df = pd.read_excel(path + '20200317_acaps_covid-19_goverment_measures_dataset_0.xlsx', sheet_name=2)

In [7]:
data_dict

,Column,Variable Name,Label,Format,Codes,Missing Values,Comments
0,A,ID,ID,Numeric,none,Not allowed,Progressive numbers of entries
1,B,COUNTRY,COUNTRY,Text,none,Not allowed,Manual
2,C,ISO,ISO,Text,iso3 code,Not allowed,Automated VLOOKUP with iso code
3,D,ADMIN_LEVEL,ADMIN_LEVEL,Text,none,Blank,if necessary
4,E,PCODE,PCODE,Text,none,Blank,if necessary
5,F,CATEGORY,CATEGORY,Text,none,Not allowed,Dropdown
6,G,MEASURE_TYPE,MEASURE_TYPE,Text,none,Not allowed,Dropdown
7,H,TARGETED_POP_GROUP,TARGETED_POP_GROUP,Text,none,Not allowed,Dropdown
8,I,COMMENTS,COMMENTS,Text,none,Blank,if necessary
9,J,DATE_IMPLEMENTED,DATE_IMPLEMENTED,Date (DD/MM/YYYY),none,Blank,Manual


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  815 non-null    int64         
 1   COUNTRY             815 non-null    object        
 2   ISO                 815 non-null    object        
 3   ADMIN_LEVEL_NAME    24 non-null     object        
 4   PCODE               0 non-null      float64       
 5   REGION              815 non-null    object        
 6   CATEGORY            815 non-null    object        
 7   MEASURE             815 non-null    object        
 8   TARGETED_POP_GROUP  777 non-null    object        
 9   COMMENTS            694 non-null    object        
 10  DATE_IMPLEMENTED    701 non-null    datetime64[ns]
 11  SOURCE              797 non-null    object        
 12  SOURCE_TYPE         793 non-null    object        
 13  LINK                792 non-null    object        

In [105]:
df.head()

,ID,COUNTRY,ISO,ADMIN_LEVEL_NAME,PCODE,REGION,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS,DATE_IMPLEMENTED,SOURCE,SOURCE_TYPE,LINK,ENTRY_DATE,Alternative source
0,1,Afghanistan,AFG,NaN,NaN,Asia,Public health measures,Health screenings in airports and border cross...,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
1,2,Afghanistan,AFG,Kabul,NaN,Asia,Public health measures,Introduction of quarantine policies,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
2,3,Afghanistan,AFG,NaN,NaN,Asia,Public health measures,Awareness campaigns,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
3,4,Afghanistan,AFG,NaN,NaN,Asia,Social and economic measures,Emergency administrative structures activated ...,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
4,5,Afghanistan,AFG,NaN,NaN,Asia,Social distancing,Limit public gatherings,No,Nevruz festival cancelled,2020-03-12,AA,Media,https://www.aa.com.tr/en/asia-pacific/coronavi...,2020-03-14,NaN


In [35]:
all_countries = df['COUNTRY'].unique()
all_countries

array(['Afghanistan', 'Bhutan', 'Antigua and Barbuda', 'Bolivia',
       'Bosnia and Herzegovina', 'Argentina', 'Albania', 'Nepal',
       'Algeria', 'Bangladesh', 'Angola', 'Chile', 'Uruguay', 'Paraguay',
       'Peru', 'Azerbaijan', 'Brazil', 'Portugal', 'Ecuador', 'Colombia',
       'Armenia', 'Bahrain', 'Venezuela', 'Guyana', 'Suriname', 'Belgium',
       'Panama', 'Australia', 'Costa Rica', 'Nicaragua', 'Honduras',
       'Belize', 'El Salvador', 'Bahamas', 'Guatemala', 'Benin', 'Mexico',
       'Brunei Darussalam', 'Bulgaria', 'Papua New Guinea',
       'Solomon Islands', 'Cameroon', 'Tuvalu', 'Vanuatu', 'Fiji',
       "Côte d'Ivoire", 'Tonga', 'Nauru', 'Kiribati', 'Indonesia',
       'Senegal', 'Philippines', 'Chad', 'Malaysia', 'Burkina Faso',
       'China', 'Congo DR', 'Barbados', 'Canada', 'Thailand',
       'New Zealand', 'Congo', 'United Kingdom', 'Comoros', 'Madagascar',
       'Cyprus', 'Dominica', 'United States of America', 'Mali',
       'Grenada', 'Saint Lucia', 'Sai

In [29]:
df['COUNTRY'].unique().shape

(186,)

In [11]:
df['CATEGORY'].unique()

array(['Public health measures', 'Social and economic measures',
       'Social distancing', 'Movement restrictions', 'Human rights'],
      dtype=object)

In [12]:
df['MEASURE'].unique()

array(['Health screenings in airports and border crossings',
       'Introduction of quarantine policies', 'Awareness campaigns',
       'Emergency administrative structures activated or established',
       'Limit public gatherings', 'Border closure\xa0',
       'Schools closure\xa0', 'Travel restrictions',
       'Strengthening the public health system', 'Flights suspension',
       'Limit product imports/exports', 'Border checks\xa0',
       'Visa restrictions', 'Checkpoints', 'Economic measures',
       'State of emergency declared',
       'Additional health/documents requirements upon arrival',
       'Public services closure\xa0',
       'Changes in prison-related policies', 'General recommendations',
       'General lockdown',
       'Psychological assistance and medical social work',
       'Obligatory medical tests not related to COVID-19',
       'Screening of all Citizens', 'Electronic surveilance',
       'Lockdown of refugee/idp camps'], dtype=object)

In [28]:
df['MEASURE'].unique().shape

(26,)

In [10]:
countries_of_int = ['China', 'Italy', 'United States of America', 'Korea Republic of']

In [33]:
for country in countries_of_int:
    print('====================================')
    print(country)
    country_mask = df['COUNTRY'] == country
    print(df[country_mask]['ID'].count() , 'measures enacted')
    print('earliest' , str(df[country_mask]['DATE_IMPLEMENTED'].min()))
    print('most recent' , str(df[country_mask]['DATE_IMPLEMENTED'].max()))
    print(df[country_mask][['DATE_IMPLEMENTED', 'MEASURE', 'CATEGORY']])
    print('\n')

China
17 measures enacted
earliest 2020-01-20 00:00:00
most recent 2020-03-19 00:00:00
    DATE_IMPLEMENTED                                            MEASURE  \
177              NaT                                   Schools closure    
178              NaT                           Public services closure    
179              NaT                Introduction of quarantine policies   
180       2020-01-29                            Limit public gatherings   
181       2020-03-14                                  Economic measures   
182              NaT                                Awareness campaigns   
183              NaT  Health screenings in airports and border cross...   
184              NaT  Emergency administrative structures activated ...   
185       2020-01-27                                Travel restrictions   
186              NaT             Strengthening the public health system   
187       2020-01-20                        State of emergency declared   
188          

Interesting, S. Korea has relatively few measure enacted for a country that has flattened the curve. However, in terms of number of tests performed, they are leading the pack. In terms of testing per capita, they are quite high as well:

https://ourworldindata.org/covid-testing#current-covid-19-test-coverage-estimates

So maybe number of tests performed should count as some kind of "countermeasure". The problem being there is no time element to this. This may require some assumptions. Will need to have a look at the data.

Perhaps, when the number of per capita tests is low, an increase leads to an increase in the number of confirmed cases. But when it is high, it leads to a decrease, as more perfect knowledge allows infected people to be quarantined and slow the spread.

---

In any case, there are a number of features I can create there, where each one is at the grain of country/date:
- Count of total measures
- Count of measures by CATEGORY
- Count of measures by MEASURE

Of these, I like the last one, as it is the most granular. XGBoost should be able to avoid overfitting here. But maybe the first one is good too, as a sort of aggregate and very simple feature.

Issues:
- Some dates are missing. Maybe I should only count the ones that have dates? Otherwise I'd have to do some kind of imputation.
- Need to get country names aligned with Kaggle data

Otherwise it seems pretty straight-forward.

---

What is the null fraction of dates?

In [38]:
na_date_mask = df['DATE_IMPLEMENTED'].isnull()
print(na_date_mask.mean(), na_date_mask.sum())

0.13987730061349693 114


How would I impute these? One option is time consuming research, although if the answer were knowable presumably the researchers would have included it. Another option is to simply drop them. This is easy, however in China for example, most of the countermeasures don't have date data:

In [32]:
df[df['COUNTRY'] == 'China'][['DATE_IMPLEMENTED', 'CATEGORY', 'MEASURE', 'TARGETED_POP_GROUP', 'COMMENTS']]

,DATE_IMPLEMENTED,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS
177,NaT,Social distancing,Schools closure,NaN,SCFN's (schools for the children of foreign na...
178,NaT,Social distancing,Public services closure,NaN,NaN
179,NaT,Public health measures,Introduction of quarantine policies,Yes,all people with symptoms and in some regions t...
180,2020-01-29,Social distancing,Limit public gatherings,No,all provincial-level regions on the Chinese ma...
181,2020-03-14,Social and economic measures,Economic measures,Yes,people with low income levels
182,NaT,Public health measures,Awareness campaigns,NaN,NaN
183,NaT,Public health measures,Health screenings in airports and border cross...,Yes,International flighes
184,NaT,Social and economic measures,Emergency administrative structures activated ...,NaN,NaN
185,2020-01-27,Movement restrictions,Travel restrictions,NaN,NaN
186,NaT,Public health measures,Strengthening the public health system,NaN,"Two hospitals, namely Huoshenshan and Leishens..."


Another option is to impute in an automated way. I think a sensible way to do this would be to select a random date within the range of the first date a country has any countermeasure, and the present.

For countries where all the date data is null, this is certainly ethnocentric of me, but I will select a random date from the first U.S. countermeasure, to the present.

In [47]:
us_start_date = datetime.datetime(2020,1,31)

What is the latest date in the dataset?

In [40]:
df['DATE_IMPLEMENTED'].max()

Timestamp('2020-03-27 00:00:00')

Interesting, this is in the future. Must be a planned measure. I'll used today's date as the last date a measure can be selected from.

In [44]:
end_date = datetime.datetime(2020, 3, 20)

In [68]:
random.seed(24)
random.choices([1, 2, 3, 4], k=3)

[3, 4, 1]

In [69]:
datetime.timedelta(days=2).days

2

https://stackoverflow.com/questions/13703720/converting-between-datetime-timestamp-and-datetime64

In [97]:
df_filled = df.copy()
random.seed(24)
for country in all_countries:
#     print('====================================')
#     print(country)
    country_mask = df['COUNTRY'] == country
#     print(df[country_mask]['DATE_IMPLEMENTED'])
    min_date_here = df[country_mask]['DATE_IMPLEMENTED'].min()
    if pd.isnull(min_date_here):
        min_date_here = us_start_date
    
    country_null_mask = country_mask & na_date_mask
    n_nulls_this_country = (country_null_mask).sum()
    if n_nulls_this_country > 0:
#         print(country)
#         print(n_nulls_this_country)
#         print(country_mask.sum())
#         print(min_date_here)
        n_days = (end_date - min_date_here).days
        days_range = list(range(1,n_days+1))
        days_choice = random.choices(days_range, k=n_nulls_this_country)
        dates_to_fill = [np.datetime64(min_date_here + datetime.timedelta(days=this_day))
                         for this_day in days_choice]
#         print(dates_to_fill)
#         print(df_filled.loc[country_null_mask, 'DATE_IMPLEMENTED'].shape)
#         print(len(dates_to_fill))
        df_filled.loc[country_null_mask, 'DATE_IMPLEMENTED'] = dates_to_fill

In [98]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  815 non-null    int64         
 1   COUNTRY             815 non-null    object        
 2   ISO                 815 non-null    object        
 3   ADMIN_LEVEL_NAME    24 non-null     object        
 4   PCODE               0 non-null      float64       
 5   REGION              815 non-null    object        
 6   CATEGORY            815 non-null    object        
 7   MEASURE             815 non-null    object        
 8   TARGETED_POP_GROUP  777 non-null    object        
 9   COMMENTS            694 non-null    object        
 10  DATE_IMPLEMENTED    815 non-null    datetime64[ns]
 11  SOURCE              797 non-null    object        
 12  SOURCE_TYPE         793 non-null    object        
 13  LINK                792 non-null    object        

In [99]:
df_filled[df_filled['COUNTRY'] == 'China']\
[['DATE_IMPLEMENTED', 'CATEGORY', 'MEASURE', 'TARGETED_POP_GROUP', 'COMMENTS']]

,DATE_IMPLEMENTED,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS
177,2020-02-09,Social distancing,Schools closure,NaN,SCFN's (schools for the children of foreign na...
178,2020-02-08,Social distancing,Public services closure,NaN,NaN
179,2020-02-22,Public health measures,Introduction of quarantine policies,Yes,all people with symptoms and in some regions t...
180,2020-01-29,Social distancing,Limit public gatherings,No,all provincial-level regions on the Chinese ma...
181,2020-03-14,Social and economic measures,Economic measures,Yes,people with low income levels
182,2020-02-12,Public health measures,Awareness campaigns,NaN,NaN
183,2020-03-14,Public health measures,Health screenings in airports and border cross...,Yes,International flighes
184,2020-03-16,Social and economic measures,Emergency administrative structures activated ...,NaN,NaN
185,2020-01-27,Movement restrictions,Travel restrictions,NaN,NaN
186,2020-02-08,Public health measures,Strengthening the public health system,NaN,"Two hospitals, namely Huoshenshan and Leishens..."


Works.

# Match up country names

In [115]:
path = '../covid19-global-forecasting-week-1/'
train = pd.read_csv(path + 'train.csv')

In [116]:
train.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0


In competition data set but not here

In [117]:
set(train['Country/Region'].unique()) - set(all_countries)

{'Andorra',
 'Aruba',
 'Brunei',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 "Cote d'Ivoire",
 'Cruise Ship',
 'Cuba',
 'Czechia',
 'French Guiana',
 'Gabon',
 'Gambia, The',
 'Greenland',
 'Guadeloupe',
 'Guam',
 'Guernsey',
 'Guinea',
 'Holy See',
 'Jersey',
 'Korea, South',
 'Kosovo',
 'Martinique',
 'Mayotte',
 'Moldova',
 'Monaco',
 'North Macedonia',
 'Puerto Rico',
 'Republic of the Congo',
 'Reunion',
 'Russia',
 'San Marino',
 'Taiwan*',
 'The Bahamas',
 'The Gambia',
 'US',
 'Vietnam'}

Here but not in competition:

In [118]:
set(all_countries) - set(train['Country/Region'].unique())

{'Angola',
 'Bahamas',
 'Belize',
 'Brunei Darussalam',
 'Burundi',
 'Cabo Verde',
 'Chad',
 'Comoros',
 'Congo',
 'Congo DR',
 'Czech Republic',
 "Côte d'Ivoire",
 'Dominica',
 'El Salvador',
 'Eritrea',
 'Fiji',
 'Gambia',
 'Grenada',
 'Guinea-Bissau',
 'Haiti',
 'Kiribati',
 'Korea DPR',
 'Korea Republic of',
 'Lao PDR',
 'Lesotho',
 'Libya',
 'Madagascar',
 'Malawi',
 'Mali',
 'Marshall Islands',
 'Micronesia',
 'Moldova Republic Of',
 'Mozambique',
 'Myanmar',
 'Nauru',
 'Nicaragua',
 'Niger',
 'North Macedonia Republic Of',
 'Palau',
 'Palestine',
 'Papua New Guinea',
 'Russian Federation',
 'Samoa',
 'Sierra Leone',
 'Solomon Islands',
 'South Sudan',
 'Syria',
 'Tajikistan',
 'Timor-Leste',
 'Tonga',
 'Turkmenistan',
 'Tuvalu',
 'Uganda',
 'United States of America',
 'Vanuatu',
 'Viet Nam',
 'Yemen',
 'Zimbabwe',
 'kenya'}

Unfortunately this needs to be done manually.

In [126]:
mapper = {'Bahamas':'The Bahamas',
         'Brunei Darussalam':'Brunei',
         'Congo':'Congo (Brazzaville)',
         'Congo DR':'Republic of the Congo',
         'Czech Republic':'Czechia',
         "Côte d'Ivoire":"Cote d'Ivoire",
         'Gambia':'The Gambia',
         'Guinea-Bissau':'Guinea',
         'Korea Republic of':'Korea, South',
         'Moldova Republic Of':'Moldova',
         'North Macedonia Republic Of':'North Macedonia',
         'Russian Federation':'Russia',
         'United States of America':'US',
         'Viet Nam':'Vietnam'}

Need to add elements in common to mapper.

In [127]:
intersection = set(all_countries) & set(train['Country/Region'].unique())
# intersection

In [128]:
for in_common_country in intersection:
    mapper[in_common_country] = in_common_country

In [129]:
df_countries_renamed = df_filled.copy()

In [130]:
df_countries_renamed['COUNTRY'] = df_countries_renamed['COUNTRY'].map(mapper)

In [131]:
set(train['Country/Region'].unique()) - set(df_countries_renamed['COUNTRY'].unique())

{'Andorra',
 'Aruba',
 'Congo (Kinshasa)',
 'Cruise Ship',
 'Cuba',
 'French Guiana',
 'Gabon',
 'Gambia, The',
 'Greenland',
 'Guadeloupe',
 'Guam',
 'Guernsey',
 'Holy See',
 'Jersey',
 'Kosovo',
 'Martinique',
 'Mayotte',
 'Monaco',
 'Puerto Rico',
 'Reunion',
 'San Marino',
 'Taiwan*'}

Here but not in competition:

In [132]:
set(df_countries_renamed['COUNTRY'].unique()) - set(train['Country/Region'].unique())

{nan}

If a country doesn't appear in the training set, its name is missing from this new DF.

Probably can copy over to `'Congo (Kinshasa)'` from the other Congo. 

# Engineer features

Want a lag time from when the change was made? Perhaps the 7 day average incubation period?

---

Want some kind of `DataFrame` that can be joined on country/date, with the competition data. How to do this? Perhaps a multi-index on country/date, with columns being the features.

In [133]:
df_countries_renamed.head()

,ID,COUNTRY,ISO,ADMIN_LEVEL_NAME,PCODE,REGION,CATEGORY,MEASURE,TARGETED_POP_GROUP,COMMENTS,DATE_IMPLEMENTED,SOURCE,SOURCE_TYPE,LINK,ENTRY_DATE,Alternative source
0,1,Afghanistan,AFG,NaN,NaN,Asia,Public health measures,Health screenings in airports and border cross...,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
1,2,Afghanistan,AFG,Kabul,NaN,Asia,Public health measures,Introduction of quarantine policies,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
2,3,Afghanistan,AFG,NaN,NaN,Asia,Public health measures,Awareness campaigns,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
3,4,Afghanistan,AFG,NaN,NaN,Asia,Social and economic measures,Emergency administrative structures activated ...,No,NaN,2020-02-12,Ministry of Health,Government,https://moph.gov.af/en/moph-held-emergency-mee...,2020-03-14,NaN
4,5,Afghanistan,AFG,NaN,NaN,Asia,Social distancing,Limit public gatherings,No,Nevruz festival cancelled,2020-03-12,AA,Media,https://www.aa.com.tr/en/asia-pacific/coronavi...,2020-03-14,NaN


Maybe: Some kind of measure of how compliant citizenry is?